# Decoder masked multi-head attention

Una vez explicado el `Scaled Dot-Product Attention` con enmascaramiento podemos volver al `Masked Multi-Head Attention`

<div style="text-align:center;">
  <img src="Imagenes/transformer_architecture_model_decoder_masked_multi_head_attention.png" alt="Multi-Head Attention" style="width:425px;height:626px;">
  <img src="Imagenes/multi-head_attention.png" alt="Multi-Head Attention" style="width:501px;height:623px;">
</div>

Si recordamos cuando vimos el `Multi-Head Attention` del encoder, lo que hace el transformer es poner unas capas `Linear` entre `Q`, `K` y `V` y el `Scaled Dot-Product Attention`. Estas capas `Linear` son unas capas fully connected que lo que harán será quedarse con una parte de las distintas dimensiones del embedding, ya que tendrá mejor efecto calcular el `Scaled Dot-Product Attention` solo entre unas dimensiones que tengan características similares y no entre todas las dimensiones.

Estas capas `Linear` son redes neuronales de manera que sus pesos se van cambiando durante el entrenamiento para que se junten las dimensiones del embedding que mejor funcionan juntas

Gracias a estas capas `Linear` el espacio de embedding se divide en `h` grupos, donde `h` es un hiperparámetro que elegimos nosotros

Después se concatenan las matrices resultantes del `Scaled Dot-Product Attention` en la capa `Concat`, para volver a juntar toda la información

Y por último, como con la concatenación se vamos a tener una matriz de BSx(h·num_tokens)xdim_embedding, necesitamos pasar esta matriz por una última capa `Linear` para tener una matriz de dimensiones BSxnum_tokensxdim_embedding para que comprima esta información

## Implementación

Ya hicimos una clase para esto, así que la recuperamos

In [2]:
import torch
import torch.nn as nn
import math

class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        self.dim_embedding = dim_embedding
    
    def forward(self, key, query, value):
        """
        Args:
            key: key vector
            query: query vector
            value: value vector
        
        Returns:
            output vector from scaled dot product attention
        """
        # MatMul
        key_trasposed = key.transpose(-1,-2)
        product = torch.matmul(query, key_trasposed)
        # scale
        scale = product / math.sqrt(self.dim_embedding)
        # softmax
        attention_matrix = torch.nn.functional.softmax(scale, dim=-1)
        # MatMul
        output = torch.matmul(attention_matrix, value)
        
        return output

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, dim_embedding):
        """
        Args:
            heads: number of heads
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        
        self.dim_embedding = dim_embedding
        self.dim_proyection = dim_embedding // heads
        self.heads = heads
        
        self.proyection_Q = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_K = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_V = nn.Linear(dim_embedding, dim_embedding)
        self.attention = nn.Linear(dim_embedding, dim_embedding)

        self.scaled_dot_product_attention = ScaledDotProductAttention(self.dim_proyection)
    
    def forward(self, Q, K, V):
        """
        Args:
            Q: query vector
            K: key vector
            V: value vector

        Returns:
            output vector from multi-head attention
        """
        batch_size = Q.size(0)
        
        # perform linear operation and split into h heads
        proyection_Q = self.proyection_Q(Q).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_K = self.proyection_K(K).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_V = self.proyection_V(V).view(batch_size, -1, self.heads, self.dim_proyection)
        
        # transpose to get dimensions bs * h * sl * d_model
        proyection_Q = proyection_Q.transpose(1,2)
        proyection_K = proyection_K.transpose(1,2)
        proyection_V = proyection_V.transpose(1,2)

        # calculate attention
        scaled_dot_product_attention = self.scaled_dot_product_attention(proyection_Q, proyection_K, proyection_V)
        
        # concatenate heads and put through final linear layer
        concat = scaled_dot_product_attention.transpose(1,2).contiguous().view(batch_size, -1, self.dim_embedding)
        
        output = self.attention(concat)
    
        return output